# 02 - preprocesado

En este notebook vamos a realizar el preprocesado de los datos que obtenemos del `train.csv` de la competencia de Kaggle Udea/ai4eng 20252.

Este preprocesado es necesario con el fin de 'normalizar' los distintos datos que existen en el `train.csv` ya sea porque estan en un formato nulo o simplemente para facilitar la posterior creacion del modelo.

El objetivo final es crear una función de preprocesado flexible que nos permita probar distintas estrategias (imputación, escalado, codificación) fácilmente en la siguiente entrega.

### Realizamos importaciones iniciales y cargamos los datos

In [ ]:
import pandas as pd
import numpy as np
import os, json
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import userdata
from unidecode import unidecode

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
user = userdata.get('KAGGLE_USERNAME')
key = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = user
os.environ["KAGGLE_KEY"] = key
assert user and key, "Faltan los secretos KAGGLE_USERNAME/KAGGLE_KEY"
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.08GB/s]
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [ ]:
z = pd.read_csv("train.csv")
z.head(5)

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,Menos de 10 horas,Estrato 5,Si,Educación profesional completa,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,medio-alto,0.553,0.142,0.248,0.175
6,47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Educación profesional incompleta,Si,...,N,Si,Si,Si,Técnica o tecnológica completa,alto,0.242,0.299,0.267,0.308
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Si,...,N,Si,No,Si,Secundaria (Bachillerato) incompleta,medio-bajo,0.238,0.316,0.286,0.290
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Educación profesional incompleta,medio-bajo,0.147,0.407,0.212,0.328
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 5,Si,Postgrado,Si,...,N,Si,Si,Si,Postgrado,alto,0.535,0.122,0.257,0.167


## Limpieza inicial de datos duplicados y texto

Primero, revisamos la información básica y eliminamos columnas duplicadas.

In [ ]:
z.info()

In [ ]:
if "F_TIENEINTERNET.1" in z.columns:
    z = z.drop(columns=["F_TIENEINTERNET.1"])

Ahora, limpiemos columnas de texto, de este modo unificamos categorias y evitamos duplicados.

In [ ]:
def clean_text(text):
  if not isinstance(text, str):
    return text

  text = unidecode(text).upper() # Quitar tildes y pasar a mayúsculas

  # Correcciones manuales
  corrections = {
    'INGENIER?A': 'INGENIERIA',
    'ADMINISTRACI?N': 'ADMINISTRACION',
    'COMUNICACI?N': 'COMUNICACION',
    'DISE?O': 'DISENO',
    'QU?MICA': 'QUIMICA',
    'FILOSOF?A': 'FILOSOFIA',
    'ECONOM?A': 'ECONOMIA'
  }

  for wrong, right in corrections.items():
    text = text.replace(wrong, right)
  return text

# Aplicar limpieza a las columnas de texto relevantes que e identificado de momento
text_cols_to_clean = ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']

for col in text_cols_to_clean:
  if col in z.columns:
    z[col] = z[col].apply(clean_text)

print("Columnas de texto limpiadas.")
print("\nValores únicos de E_PRGM_ACADEMICO (ejemplo):")
print(z['E_PRGM_ACADEMICO'].value_counts().head())

## Limpieza y Codificacion para variable objetivo

Nuestra variable objetivo `RENDIMIENTO_GLOBAL` es categórica. Debemos asegurarnos de que solo contenga los valores válidos y luego codificarla numéricamente para el análisis de correlación y el entrenamiento.

In [ ]:
print(f"Valores únicos antes de limpiar: {z['RENDIMIENTO_GLOBAL'].unique()}")

# Filtrar solo las categorías válidas
valid_categories = ['alto', 'medio-alto', 'medio-bajo', 'bajo']
z = z[z['RENDIMIENTO_GLOBAL'].isin(valid_categories)].copy()

print(f"\nValores únicos después de limpiar: {z['RENDIMIENTO_GLOBAL'].unique()}")

# Definir un mapeo ordinal para la variable objetivo
target_map = {
  'bajo': 0,
  'medio-bajo': 1,
  'medio-alto': 2,
  'alto': 3
}

# Crear la columna 'Y_NUMERIC' para el análisis
z['Y_NUMERIC'] = z['RENDIMIENTO_GLOBAL'].map(target_map)

## Análisis de Correlación

Ahora que tenemos una variable objetivo numérica (`Y_NUMERIC`), podemos ver cómo se correlaciona con las otras variables numéricas del dataset.

In [ ]:
# Seleccionar columnas numéricas y la nueva variable objetivo
numerical_features = ['PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4', 'Y_NUMERIC']
corr_matrix = z[numerical_features].corr()

# Graficar el heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Heatmap de Correlación (Features Numéricos vs. Rendimiento)')
plt.show()

In [ ]:
z.info()
z.describe()
z.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

,0
ID,0
PERIODO_ACADEMICO,0
E_PRGM_ACADEMICO,0
E_PRGM_DEPARTAMENTO,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENELAVADORA,39773


In [ ]:
NA = z.isna().mean().sort_values(ascending=False)
NA.head(30)

,0
F_TIENEAUTOMOVIL,0.062994
F_TIENELAVADORA,0.057434
F_TIENECOMPUTADOR,0.055022
F_ESTRATOVIVIENDA,0.046407
E_HORASSEMANATRABAJA,0.044559
F_TIENEINTERNET,0.038453
F_EDUCACIONMADRE,0.034172
F_EDUCACIONPADRE,0.033470
E_PAGOMATRICULAPROPIO,0.009383
E_VALORMATRICULAUNIVERSIDAD,0.009079


Ahora si vamos a manejar los datos nulos, mas sin embargo existen distintas estrategias para combatir estos valores nulos, podemos eliminar las filas con valores nulos, ponerlos aplicar media, mediana etc. mas sin embargo estos manejos pueden influir de buena o mala manera para el entrenamiento del modelo, por lo que mas adelante en la entrega #3 probare como se comporta el modelo al interactuar con las distintas formas de manejar estos valores nulos con el fin de encontrar la más adecuada.

### Eliminacion de filas con valores nulos

In [ ]:
z_copy_delet_null = z.copy()
z_copy_delet_null.dropna(subset=["F_TIENEAUTOMOVIL", "F_TIENELAVADORA", "F_TIENECOMPUTADOR", "F_ESTRATOVIVIENDA", "E_HORASSEMANATRABAJA", "F_TIENEINTERNET", "F_EDUCACIONMADRE", "F_EDUCACIONPADRE", "E_PAGOMATRICULAPROPIO", "E_VALORMATRICULAUNIVERSIDAD"], inplace=True)
z_copy_delet_null

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto,0.237,0.271,0.271,0.311
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,N,No,Si,Secundaria (Bachillerato) incompleta,bajo,0.314,0.240,0.278,0.260
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo,0.286,0.240,0.314,0.287
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,No,N,Si,Si,Primaria completa,bajo,0.132,0.426,0.261,0.328


In [ ]:
print("El número de filas eliminadas es:", len(z) - len(z_copy_delet_null))

El número de filas eliminadas es: 63604


Vemos que es una cantidad considerable, lo que podria probocar que no sea la mejor opcion

### Ahora vamos a intentar con otro metodo rellenando los nulos, sin embargo para ello antes debemos organizar las distintas categorias

In [ ]:
target_col = 'RENDIMIENTO_GLOBAL'
id_col = 'ID'

Y = z[target_col] if target_col in z.columns else None
X = z.drop(columns=[c for c in [target_col, id_col] if c in z.columns])
X.head(2)

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,0.322,0.208,0.310,0.267
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,0.311,0.215,0.292,0.264


In [ ]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

num_cols, cat_cols[:10]

(['PERIODO_ACADEMICO',
  'INDICADOR_1',
  'INDICADOR_2',
  'INDICADOR_3',
  'INDICADOR_4'],
 ['E_PRGM_ACADEMICO',
  'E_PRGM_DEPARTAMENTO',
  'E_VALORMATRICULAUNIVERSIDAD',
  'E_HORASSEMANATRABAJA',
  'F_ESTRATOVIVIENDA',
  'F_TIENEINTERNET',
  'F_EDUCACIONPADRE',
  'F_TIENELAVADORA',
  'F_TIENEAUTOMOVIL',
  'E_PRIVADO_LIBERTAD'])

In [ ]:
maps = {
    'E_VALORMATRICULAUNIVERSIDAD': [
        'Entre 500 mil y menos de 1 millón',
        'Entre 1 millón y menos de 2.5 millones',
        'Entre 2.5 millones y menos de 4 millones',
        'Entre 4 millones y menos de 5.5 millones',
        'Entre 5.5 millones y menos de 7 millones',
        'Más de 7 millones'
    ],
    'E_HORASSEMANATRABAJA': [
        '0',
        'Menos de 10 horas',
        'Entre 11 y 20 horas',
        'Entre 21 y 30 horas',
        'Más de 30 horas'
    ],
    'F_ESTRATOVIVIENDA': [
        'Estrato 1','Estrato 2','Estrato 3','Estrato 4','Estrato 5','Estrato 6','Estrato 0'
    ]
}

Para escoger estrategia podemos probar y/o cambiar luego de manera mas facil

In [ ]:
NUM_IMPUTE = 'median' # 'mean' / 'median' / 'most_frequent' / 'constant'
CAT_IMPUTE = 'most_frequent' # típica para categóricas
ENCODING   = 'onehot' # 'onehot' o 'ordinal'
SCALER     = 'standard' # 'standard' / 'minmax' / 'robust' / 'none'

In [ ]:
def make_scaler(kind='standard'):
  if kind == 'standard': return StandardScaler()
  if kind == 'minmax': return MinMaxScaler()
  if kind == 'robust': return RobustScaler()
  return 'passthrough'

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy=NUM_IMPUTE)),
    ('scaler', make_scaler(SCALER))
])

if ENCODING == 'ordinal':
    # construimos categorías ordenadas si están definidas en el mapa de antes
    categories = []
    for c in cat_cols:
        if c in maps:
            # añade categorías no encontradas en el mapa
            known = maps[c]
            extras = [u for u in X[c].dropna().unique().tolist() if u not in known]
            categories.append(known + extras)
        else:
            categories.append(sorted(X[c].dropna().unique().tolist()))
    cat_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=CAT_IMPUTE)),
        ('encoder', OrdinalEncoder(categories=categories,
                                   handle_unknown='use_encoded_value',
                                   unknown_value=np.nan))
    ])
else:
    cat_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=CAT_IMPUTE)),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ],
    remainder='drop',
    sparse_threshold=0.3
)

X_pre = preprocessor.fit_transform(X)
X_pre.shape


(692500, 1040)

Perfecto una vez probamos esto de manera manual lo voy a trasformar en una funcion con el objetivo de poder hacer el preprocesado de la informacion de distintas formas y/o madenaras de modo que luego podamos probarlo mucho más facil a la hora de entrenar el modelo

In [ ]:
def build_preprocessor(
    df: pd.DataFrame,
    y_col: str = 'RENDIMIENTO_GLOBAL',
    id_col: str = 'ID',
    drop_rows_na: bool = False,
    drop_cols_na: bool = False,
    drop_cols_na_threshold: float | None = None,
    num_impute: str = 'median',
    cat_impute: str = 'most_frequent',
    encode_categorical: str = 'onehot',
    ordinal_maps: dict[str, list[str]] | None = None,
    scale_numeric: str = 'standard'
):
  data = df.copy()

  if "F_TIENEINTERNET.1" in data.columns:
    data = data.drop(columns=["F_TIENEINTERNET.1"])

  if drop_cols_na_threshold is not None:
    too_na = data.isnull().mean() > drop_cols_na_threshold
    data = data.drop(columns=data.columns[too_na])

  y = data.pop(y_col) if y_col in data.columns else None
  if id_col in data.columns:
    data = data.drop(columns=[id_col])

  if drop_rows_na:
    data = data.dropna(axis=0)

  if drop_cols_na:
    data = data.dropna(axis=1)

  num_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
  cat_cols = data.select_dtypes(include=['object']).columns.tolist()

  num_pipe = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy=num_impute)),
      ('scaler', make_scaler(scale_numeric))
  ])

  if encode_categorical == 'ordinal':
    maps = ordinal_maps
    categories = []
    for c in cat_cols:
      if c in maps:
        known = maps[c]
        extras = [u for u in data[c].dropna().unique().tolist() if u not in known]
        categories.append(known + extras)
      else:
        categories.append(sorted(data[c].dropna().unique().tolist()))
    cat_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('encoder', OrdinalEncoder(categories=categories, handle_unknown='use_encoded_value', unknown_value=np.nan))
    ])
  else:
    cat_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

  preproc = ColumnTransformer(
      transformers=[
          ('num', num_pipe, num_cols),
          ('cat', cat_pipe, cat_cols)
      ],
      remainder='drop',
      sparse_threshold=0.3
  )

  return preproc, data, y, num_cols, cat_cols

In [ ]:
preproc, X_feat, y_feat, num_cols_, cat_cols_ = build_preprocessor(
    z,
    y_col='RENDIMIENTO_GLOBAL',
    id_col='ID',
    drop_rows_na=False,
    drop_cols_na=False,
    drop_cols_na_threshold=None, # o 0.4 por ejemplo
    num_impute='median',
    cat_impute='most_frequent',
    encode_categorical='onehot', # prueba 'ordinal' también
    ordinal_maps=maps,
    scale_numeric='standard'
)

X_ready = preproc.fit_transform(X_feat)
X_ready.shape


(692500, 1040)